# Adatom Defect on Graphene surface

## 0. Introduction

This tutorial demonstrates creation of metal adatom on graphene hollow site.

> **Chan, K. T., Neaton, J. B., & Cohen, M. L. (2008).** 
> First-principles study of metal adatom adsorption on graphene. Physical Review B, 77(23). 
> [DOI:10.1103/physrevb.77.235430 ](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.77.235430)
    

Below is the figure 1 from the manuscript that shows the adsorption of metal adatom on graphene hollow site.


![Fig 1.](https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/defects/defect-surface-adatom-graphene/me_adatom_on_hollow_graphene.webp)



## 1. Prepare the Environment
### 1.1. Set up defects parameters 
Adatom Defect Configuration parameters are described in [Adatom Defect Configuration](https://github.com/Exabyte-io/made/blob/8196b759242551c77d1791bf5bd2f4150763cfef/src/py/mat3ra/made/tools/build/defect/configuration.py#L141).


In [ ]:
# Name of material to find in Standata
MATERIAL_NAME = "Graphene"
XY_SUPERCELL_MATRIX = [[4, 0], [0, 4]]

PLACEMENT_METHOD = "exact_coordinate"

HOLLOW_SITE_COORDINATE = [1 / 2, 1 / 2]  # in crystal coordinates
BRIDGE_SITE_COORDINATE = [3 / 8, 1 / 2]  # in crystal coordinates
TOP_SITE_COORDINATE = [7 / 12, 5 / 12]  # in crystal coordinates

# Setting data according to Table 1 in the manuscript, distance_z is the `h` from the table.
# Uncomment the lines to create the defect for the desired adatom.
ADATOM_PARAMETERS = [
    {"chemical_element": "Li", "distance_z": 1.71, "position_on_surface": HOLLOW_SITE_COORDINATE},
    {"chemical_element": "Na", "distance_z": 2.28, "position_on_surface": HOLLOW_SITE_COORDINATE},
    {"chemical_element": "K", "distance_z": 2.60, "position_on_surface": HOLLOW_SITE_COORDINATE},
    # {"chemical_element": "Ca", "distance_z": 2.29, "position_on_surface": HOLLOW_SITE_COORDINATE},
    # {"chemical_element": "Al", "distance_z": 2.13, "position_on_surface": HOLLOW_SITE_COORDINATE},
    # {"chemical_element": "Ga", "distance_z": 2.20, "position_on_surface": HOLLOW_SITE_COORDINATE},
    # {"chemical_element": "In", "distance_z": 2.45, "position_on_surface": HOLLOW_SITE_COORDINATE},
    {"chemical_element": "Sn", "distance_z": 2.82, "position_on_surface": TOP_SITE_COORDINATE},
    # {"chemical_element": "Ti", "distance_z": 1.80, "position_on_surface": HOLLOW_SITE_COORDINATE},
    # {"chemical_element": "Fe", "distance_z": 1.53, "position_on_surface": HOLLOW_SITE_COORDINATE},
    {"chemical_element": "Pd", "distance_z": 2.21, "position_on_surface": BRIDGE_SITE_COORDINATE},
    # {"chemical_element": "Au", "distance_z": 2.69, "position_on_surface": TOP_SITE_COORDINATE},
]

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("https://exabyte-io.github.io/api-examples/mat3ra_api_examples-0.1.dev1+g03efa43-py3-none-any.whl", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples|create_point_defect.ipynb")

### 1.3. Load input material

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

material = Material.create(Materials.get_by_name_first_match(MATERIAL_NAME))

### 1.4. Create and preview Supercell

In [ ]:
from mat3ra.made.tools.helpers import create_slab
from utils.visualize import visualize_materials as visualize

supercell = create_slab(crystal=material,
                        miller_indices=(0, 0, 1),
                        xy_supercell_matrix=XY_SUPERCELL_MATRIX
                        )
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create the Defect

In [ ]:
from mat3ra.made.tools.build.defect.adatom.helpers import create_adatom_defect

materials_with_adatom = [create_adatom_defect(
    slab=supercell,
    placement_method=PLACEMENT_METHOD,
    element=adatom_parameters["chemical_element"],
    distance_z=adatom_parameters["distance_z"],
    position_on_surface=adatom_parameters["position_on_surface"],
) for adatom_parameters in ADATOM_PARAMETERS]

## 3. Visualize Result(s)

In [ ]:
from utils.visualize import visualize_materials as visualize

for material_with_defect in materials_with_adatom:
    title = f"{material_with_defect.name} adatom"
    visualize([{"material": material_with_defect, "title": title}], viewer="wave")

## 4. Download the Results

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials(materials_with_adatom)
for material_with_defect in materials_with_adatom:
    download_content_to_file(material_with_defect.to_json(),
                             f"{material_with_defect.name}.json")